# Chapter 5 - Dictionaries and Structuring Data

In [453]:
# Let's see what methods are available for Powershell HashTables
!powershell @{} ^| Get-Member -MemberType Method



   TypeName: System.Collections.Hashtable

Name              MemberType Definition                                        
----              ---------- ----------                                        
Add               Method     void Add(System.Object key, System.Object value...
Clear             Method     void Clear(), void IDictionary.Clear()            
Clone             Method     System.Object Clone(), System.Object ICloneable...
Contains          Method     bool Contains(System.Object key), bool IDiction...
ContainsKey       Method     bool ContainsKey(System.Object key)               
ContainsValue     Method     bool ContainsValue(System.Object value)           
CopyTo            Method     void CopyTo(array array, int arrayIndex), void ...
Equals            Method     bool Equals(System.Object obj)                    
GetEnumerator     Method     System.Collections.IDictionaryEnumerator GetEnu...
GetHashCode       Method     int GetHashCode()                             

## Dictionaries / HashTables

In [275]:
myCat = {'size': 'fat', 'color': 'gray', 'disposition': 'loud'}
myCat['size']

'fat'

In [284]:
!powershell ;\
$MyCat = @{size= 'fat'; color= 'gray'; disposition= 'loud'} ;\
$MyCat['size']

fat


### keys(), values(), and items() Methods

In [285]:
myCat = {'size': 'fat', 'color': 'gray', 'disposition': 'loud'}
print(myCat.keys())
print(myCat.values())
print(myCat.items())

dict_keys(['size', 'color', 'disposition'])
dict_values(['fat', 'gray', 'loud'])
dict_items([('size', 'fat'), ('color', 'gray'), ('disposition', 'loud')])


In [324]:
# Not quite sure yet how to deal with the .items / GetEnumerator yet...
!powershell ;\
$MyCat = @{size= 'fat'; color= 'gray'; disposition= 'loud'} ;\
Write-Host $MyCat.keys ;\
Write-Host $MyCat.values ;\
Write-Host ($MyCat.GetEnumerator() ^| ForEach-Object {$_.key, $_.value})

color disposition size
gray loud fat
color gray disposition loud size fat


### Checking Whether a Key or Value Exists in a Dictionary

In [315]:
myCat = {'size': 'fat', 'color': 'gray', 'disposition': 'loud'}
print('color' in myCat.keys())
print('gray' in myCat.keys())
print('gray' in myCat.values())
print('age' not in myCat.keys())

True
False
True
True


In [318]:
!powershell \
$MyCat = @{'size'='fat'; 'color'='gray'; 'disposition'= 'loud'} ;\
Write-Host ($MyCat.keys -contains 'color') ;\
Write-Host ($MyCat.keys -contains 'gray') ;\
Write-Host ($MyCat.values -contains 'gray') ;\
Write-Host (!($MyCat.keys -contains 'age'))

True
False
True
True


### Get and Setdefault behavior

In [326]:
picnicItems = {'apples': 5, 'cups': 2}
print(f'I will bring {picnicItems.get("apples", 0)} apples')
print(f'I will bring {picnicItems.get("bananas", 0)} bananas')      

I will bring 5 apples
I will bring 0 bananas


In [340]:
# Powershell doesn't have same get behavior, you can add by adding a function
!powershell \
$PicnicItems = @{apples=5; cups=2} ;\
function DictGet($a, $b) { if ($a -ne $null) { $a } else { $b } } ;\
Write-Host "I will bring (DictGet $PicnicItems['apples'] 0) apples" ;\
Write-Host "I will bring (DictGet $PicnicItems['bananas'] 0) bananas" 


I will bring 5 apples
I will bring 0 bananas


### Experimentation time!!! Create a subclass
<pre>This is nice, it allows to have similar functionality, but it's not very Pythonic!
I'd love to add a method to the HashTable class to allow similar functionality....
Let's do it!</pre>

<pre>We can create a custom class that inherets from hashtable and add Get;

This will create a new class called Dict, adding the Get method
However this is only accessible in the session it's created in. To save for future sessions,
We can save this script inside a Profile. I've done that in the $profile.CurrentUserAllHosts
Here is code to add a new profile if one doesn't already exist:

If (!(Test-Path -Path $profile.CurrentUserAllHosts)) {New-Item -ItemType File -Path $profile.CurrentUserAllHosts -Force}

</pre>

In [461]:
# Here we are saving a script to a file called dict.ps1 and running it. 
# Note this will only work if you've set your permissions to allow running scripts
script = """
Class Dict : Hashtable{
   [System.Object[]] Get ([system.object]$Key,[System.Object[]]$Values) {
       if( -not ($this.get_Keys() -contains $Key) ) {
           Return $Values
       }
       Return $this[$key]
   }
   Dict($Hash) : base($Hash) {}  
}

$PicnicItems = [Dict]@{apples=5}
Write-Host "I will bring" ($PicnicItems.Get('apples', 0)) "apples" 
Write-Host "I will bring" ($PicnicItems.Get('bananas', 0)) "bananas" 
"""

with open('dict.ps1', 'w') as f:
    f.write(script)
    
!powershell ./dict.ps1 

I will bring 5 apples
I will bring 0 bananas


In [458]:
# After adding the script to my profile, let's see if the new method is available:
# Check to see if new method exists in the Dict class!
!powershell [Dict]::new() ^| Get-Member -Name Get



   TypeName: Dict

Name MemberType Definition                                                    
---- ---------- ----------                                                    
Get  Method     System.Object[] Get(System.Object Key, System.Object[] Values)




In [382]:
# After adding the Dict class to my profile, I can just create a new type of Dict!
!powershell \
$PicnicItems = [Dict]::new() ;\
$PicnicItems.add('apples', '5') ;\
Write-Host "I will bring ($PicnicItems.Get('apples', 0)) apples" ;\
Write-Host "I will bring ($PicnicItems.Get('bananas', 0)) bananas" 

I will bring 5 apples
I will bring 0 bananas


### Let's do something similar with setdefault

In [385]:
# SetDefault in Python allows you to set a default value if none exists, but does not overwrite the value if it does exist
spam = {'name': 'Pooka', 'age': 5}
spam.setdefault('color', 'black')
print(spam['color'])
spam.setdefault('color', 'purple') #doesnt change because already has value of black
print(spam['color'])
spam['color'] = 'purple'
print(spam['color'])

black
black
purple


In [389]:
# Note in Powershell you can call a HashTable value by the hash["key"] or by hash.key
!powershell \
function DictSetDefault($dict, $key, $val) { if ($dict[$key] -eq $null) {$dict[$key]=$val }  } ;\
$Spam = @{name='Pooka'; age=5} ;\
DictSetDefault $Spam 'color' 'black' ;\
Write-Host $Spam.color  ;\
DictSetDefault $Spam 'color' 'purple' ;\
Write-Host $Spam.color ;\
$Spam['color'] = 'purple' ;\
Write-Host $Spam.color

black
black
purple


### Update our Dict class
New script is:
<pre>Class Dict : Hashtable{
   [System.Object[]] Get ([system.object]$Key, [System.Object[]]$Values) {
       if( -not ($this.get_Keys() -contains $Key) ) {
           Return $Values
       }
       Return $this[$key]
   }
   [Void] SetDefault ([system.object]$Key, [System.Object]$Values) {
       if( $this[$Key] -eq $Null ) {
           $this[$Key] = $Values
       }
   }
   Dict($Hash) : base($Hash) {}  
}
</pre>

In [464]:
# Check to see if new method exists in the Dict class!
!powershell [Dict]@{} ^| Get-Member -Name SetDefault



   TypeName: Dict

Name       MemberType Definition                                              
----       ---------- ----------                                              
SetDefault Method     void SetDefault(System.Object Key, System.Object Values)




In [462]:
# Note in Powershell you can call a HashTable value by the hash["key"] or by hash.key
!powershell \
$Spam = [Dict]@{name='pooka'; age=5} ;\
$Spam.SetDefault('color', 'black') ;\
Write-Host $Spam.color  ;\
$Spam.SetDefault('color', 'purple') ;\
Write-Host $Spam.color ;\
$Spam['color'] = 'purple' ;\
Write-Host $Spam.color

black
black
purple


### Prettify output

In [451]:
import pprint
message = 'It was a bright day in April, and the clocks were striking thirteen.'
count = {}

for character in message:
    count.setdefault(character, 0)
    count[character] = count[character] + 1
print('UGLY')
print(count)
print()
print('PRETTY')
pprint.pprint(count)

UGLY
{'I': 1, 't': 6, ' ': 12, 'w': 2, 'a': 4, 's': 3, 'b': 1, 'r': 5, 'i': 6, 'g': 2, 'h': 3, 'd': 2, 'y': 1, 'n': 4, 'A': 1, 'p': 1, 'l': 2, ',': 1, 'e': 5, 'c': 2, 'o': 1, 'k': 2, '.': 1}

PRETTY
{' ': 12,
 ',': 1,
 '.': 1,
 'A': 1,
 'I': 1,
 'a': 4,
 'b': 1,
 'c': 2,
 'd': 2,
 'e': 5,
 'g': 2,
 'h': 3,
 'i': 6,
 'k': 2,
 'l': 2,
 'n': 4,
 'o': 1,
 'p': 1,
 'r': 5,
 's': 3,
 't': 6,
 'w': 2,
 'y': 1}


In [450]:
!powershell \
$Message = 'It was a bright cold day in April, and the clocks were striking thirteen.' ;\
$Count = [Dict]::new() ;\
ForEach ($Char in $Message.toCharArray()) \
{ \
    $Count.SetDefault([string]$Char, 0); \
    $Count[[string]$Char] += 1   \
}  ;\
Write-Host "UGLY" ;\
$Count.GetEnumerator() ;\
Write-Host "PRETTY" ;\
$Count ^| ConvertTo-Json

UGLY

Name                           Value                                           
----                           -----                                           
c                              3                                               
A                              1                                               
a                              4                                               
g                              2                                               
d                              3                                               
e                              5                                               
b                              1                                               
.                              1                                               
y                              1                                               
r                              5                                               
s                              3  